In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
deploy_rh = pd.read_csv('datasets/deploy_turnover.csv',header=0)
deploy_rh

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,tenure,gender
0,221,engineering,0,0.932868,4,0,low,0.829896,5.0,M
1,232,support,0,0.718477,3,0,low,0.834544,2.0,M
2,184,sales,0,0.788830,3,0,medium,0.834988,3.0,F
3,206,sales,0,0.575688,4,0,low,0.424764,2.0,M
4,249,sales,0,0.845217,3,0,low,0.779043,3.0,M
...,...,...,...,...,...,...,...,...,...,...
995,130,finance,0,0.419646,2,0,low,0.386689,3.0,F
996,195,sales,0,0.711928,6,0,medium,0.108071,2.0,F
997,174,sales,0,0.977468,2,0,low,0.789362,3.0,F
998,159,sales,0,0.711912,4,0,low,0.668489,2.0,F


In [3]:
X = deploy_rh.copy()

In [4]:
le = preprocessing.LabelEncoder()

colunas = ["department","salary","gender"]

df_col=list(colunas)

for i in range(len(df_col)):
    X[df_col[i]] = le.fit_transform(X[df_col[i]])
    
X.head()

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,tenure,gender
0,221,2,0,0.932868,4,0,1,0.829896,5.0,1
1,232,9,0,0.718477,3,0,1,0.834544,2.0,1
2,184,8,0,0.788830,3,0,2,0.834988,3.0,0
3,206,8,0,0.575688,4,0,1,0.424764,2.0,1
4,249,8,0,0.845217,3,0,1,0.779043,3.0,1


In [5]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [6]:
modelo = pickle.load(open('grid_svm.sav', 'rb'))
modelo.predict(X)
modelo.predict_proba(X)

array([[0.05988069, 0.94011931],
       [0.9721104 , 0.0278896 ],
       [0.99238762, 0.00761238],
       ...,
       [0.83757552, 0.16242448],
       [0.99012792, 0.00987208],
       [0.08987001, 0.91012999]])

In [7]:
resultado_svm = modelo.predict(X)
resultado_svm_prob = modelo.predict_proba(X)
svm_final = pd.DataFrame(np.hstack((resultado_svm.reshape(-1, 1), resultado_svm_prob)))
svm_final.rename(columns={0: "status", 1: "% Employed", 2:"% Left"},inplace=True)

In [8]:
deploy_rh = pd.concat([deploy_rh, svm_final], axis=1, join="inner")
deploy_rh.head()

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,tenure,gender,status,% Employed,% Left
0,221,engineering,0,0.932868,4,0,low,0.829896,5.0,M,Left,0.0598807,0.940119
1,232,support,0,0.718477,3,0,low,0.834544,2.0,M,Employed,0.97211,0.0278896
2,184,sales,0,0.788830,3,0,medium,0.834988,3.0,F,Employed,0.992388,0.00761238
3,206,sales,0,0.575688,4,0,low,0.424764,2.0,M,Employed,0.990815,0.00918501
4,249,sales,0,0.845217,3,0,low,0.779043,3.0,M,Employed,0.986811,0.0131891


In [9]:
# Exportando a Base com a previsão dos resultados 
deploy_rh.to_csv('datasets/deploy_rh_final.csv', index=False)